In [1]:
#Building a NLP NMT English to Spanish using RNN technique as well as Attention mechanism

In [1]:
import tensorflow as tf
from pathlib import Path


url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng")/"spa.txt").read_text()



In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
#Now we shall remove some specific tokens present only in spanish language like inverted exclamation marks, etc
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists
pairs

[['How boring!', 'Qué aburrimiento!'],
 ['I love sports.', 'Adoro el deporte.'],
 ['Would you like to swap jobs?',
  'Te gustaría que intercambiemos los trabajos?'],
 ['My mother did nothing but weep.', 'Mi madre no hizo nada sino llorar.'],
 ['Croatia is in the southeastern part of Europe.',
  'Croacia está en el sudeste de Europa.'],
 ['I have never eaten a mango before.', 'Nunca he comido un mango.'],
 ['Tell the taxi driver to drive faster.',
  'Decile al taxista que maneje más rápido.'],
 ['Tom and I work together.', 'Tom y yo trabajamos juntos.'],
 ['I would prefer an honorable death.', 'Preferiría una muerte honorable.'],
 ['Tom married a much younger woman.',
  'Tom se ha casado con una mujer mucho más joven.'],
 ["It couldn't happen to me.", 'A mí no podría pasarme.'],
 ["Tom isn't going to marry you.", 'Tom no se va a casar contigo.'],
 ["Can you believe it? He's even lazier than me.",
  'Puedes creerlo? Él es aún más flojo que yo.'],
 ['She has hundreds of books.', 'Ella tie

In [6]:
#Creating Textvectorization layets, one per language
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length
)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [7]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [8]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [9]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [10]:
tf.random.set_seed(42)
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [11]:
#Encoding the sentences and embedding layer for each language
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [12]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [13]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [14]:
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

In [15]:
output_layer

In [16]:
#Lets create a model now
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 70s 19ms/step - loss: 0.4195 - accuracy: 0.4205 - val_loss: 0.3150 - val_accuracy: 0.5150
Epoch 2/10
3125/3125 [==============================] - 60s 19ms/step - loss: 0.2725 - accuracy: 0.5636 - val_loss: 0.2452 - val_accuracy: 0.5953
Epoch 3/10
3125/3125 [==============================] - 66s 21ms/step - loss: 0.2149 - accuracy: 0.6348 - val_loss: 0.2121 - val_accuracy: 0.6408
Epoch 4/10
3125/3125 [==============================] - 67s 21ms/step - loss: 0.1810 - accuracy: 0.6804 - val_loss: 0.1965 - val_accuracy: 0.6629
Epoch 5/10
3125/3125 [==============================] - 69s 22ms/step - loss: 0.1570 - accuracy: 0.7138 - val_loss: 0.1889 - val_accuracy: 0.6735
Epoch 6/10
2428/3125 [======================>.......] - ETA: 14s - loss: 0.1368 - accuracy: 0.7444

KeyboardInterrupt: 

In [ ]:
#Now we can not use model.predict here as the decoder also expects inputs from previous predictions
#Now we shall make a fucntion to feed the decoder as well

def translate(sentence_en):
  translation = ""
  for word_idx in range(max_length):
    X = np.array([sentence_en]) #encoder input
    X_dec = np.array(["startofseq " + translation])
    y_proba = model.predict((X, X_dec))[0, word_idx]
    predicted_word_id = np.argmax(y_proba)
    predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
    if predicted_word == "endofseq":
      break
    translation += " " + predicted_word
  return translation.strip()


In [ ]:
translate("This is love.")

1/1 [==============================] - 0s 33ms/step


'esto es amor'